## Informe de solución

__EDA__

En el EDA se realizó la revisión de los cuatro datasets __contract_df__, __internet_df__, __personal_df__, __phone_df__. Dicha revisión incluyó:

* Feature engineering: al cambiar el tipo de datos a datos requeridos como de object a datetime, intiger o float.
* Homogenización de formatos en los encabezados de las columnas y su contenido.
* Busqueda de valores ausentes típicos
* Busqueda de duplícados explicitos

Al finalizar el EDA de cada dataset mencionado se concentró la información necesaria en un sólo dataser __union_df__. Sin embargo dicha unión provoco que nuevamente se tuvieran que repetir la revisión. En esta ocasión se gestionaron valores nulos que representaban a los clientes que no tenian contratado el servicio teléfonico pero si el de internet, los cuales fueron llenados con 0 para indicar la ausencia de dicho servicio mientras que los otros valores posibles eran strings para indicar 'dsl' o 'fibra_optica', ya que al realizar el preprocesamiento dicha columna categórica sería transformada mediante One-Hot Encoding y Codificación de etiquetas.

Se realizó el filtrado del DataFrame considerando las columnas de __Begin_Date__ y __End_Date__ para abarcar un intervalo de 6 años como se detalló arriba.

Finalmente se exploraron las relaciones entre las variables cuantitativas utilizando gráficos con pairplot, en función del tipo de contrato y de si los clientes en la ventada de tiempo seleccionada habian cancelado o no el servicio con Telecom.


__Preporcessing__

Se eliminó la columna __Customer_ID__ ya que no aportaria valor durante el entrenamiento y porque dicha columna se habia utilizado previamente para unir los diferentes datasets originales pero para el entrenamiento posterior carecia de importancia.
Durante el entrenamiento las columnas tipo datetime generarían problemas ya que no son identificadas por los modelos, por lo que se extrajo el valor de cada termino de la fecha mediante el método .dt creando columnas individuales para el año, mes, día y día de la semana con la finalidad de que dicha información no se perdierá y fuera considerado por el entrenamiento de los modelos. De igual forma, una vez extraída y creadas las columnas con los terminos de la fecha en variables númericas, la columna __Begin_Date__ fue eliminada. 
Se definio las características y el objetivo y posteriormente de cara a la creación de un modelo de regresión lineal se utilizó la técnica de One-Hot Encoding para transformar las columnas categóricas a columnas dummy eliminando la primera de ellas, evitando así la trampa dummy. Es así como se creo el dataset para el modelo de regresión logísgtica df_ohe.
De manera similar se utilizó la técnica de la codificación de etiquetas para transformar las columnas categóricas a númericas de cara a la creación de modelos de bosque aleatorio de clasificación, creando el dataset para lod modelos de bosque aleatorio df_ordinal. 
Después se realizó la segmentación de los datos sin considerar el parámetro stratify como se pensó en un inicio ya que las métricas resultaron ligeramente superiores al no considerarlo. Debido a que la segmentación se hizó después de la transformación de las columnas categóricas se crearon dos segmentaciones, una para el modelo de regresión logística y otra para el los modelos de bosque aleatorio. La segmententación de los datos contemplo como se señaló previamente tres conjuntos:

* Train 60%
* Valid 20%
* Test 20%

Las columnas __Total_Charge__ y __Monthly_Charges__ fueron tratadas con StandarScaler para asegurar que la diferencia en los ordenes de magnitud entre el total de cargos acumulados por un cliente y su gasto mensual no fuera mal interpretada por el modelo, esto se realizó en cada una de las dos segmentaciones mencionadas arriba (modelo de regresión logística y modelos de bosque aleatorio). La razón por la cual se utilizó posterior a la segmentación de datos fue para evitar que se manipulara el conjunto de test repercutiendo en el overfitting de los modelos.

Finalmente se trato el desequilibrio de clases en la columna target utilizando el sobremuestreo.


__Creación de modelos__

Se decidió probar una serie de modelos y con base en aquel que tuviese el mayor AUC-ROC score en el conjunto de validación se prabaría como modelo final con el conjunto de test. Al construir los modelos se fue modificando de forma controlada las modificaciones de tal forma que en algunos se utilizó la validación cruzada __(CV)__ buscando robustecer el entrenamiento de los modelos, en otros se utilizaron frameworks para la optimización de hiperparámetros, en otros se utilizó la potenciación de gradiente y finalmente se probo la combinación varias de estas combianciones. Las métricas mostradas en este apartado han sido redondeadas a dos decimales por practicidad, sin embargo el valor exacto se puede consultar en su apartado correspondiente.


* Regresión logística

 Se comenzó con la regresión logística debido a que al ser más sencillo el tiempo para su entrenamiento también sería menor.
Sin embargo, al utilizar este modelo con y sin validación cruazada no se alcanzó el valor umbral. Este modelo no permitio un gran tunning de hiperparámetros por lo que se consideraron modelos de bosques aleatorios.

Nota: En un inicio se había considerado utilizar modelos de árboes de decisión también, sin embargo, al requerir el mismo preprocesamiento de datos se prefirio utilizar únicamente los modelos de bosques aleatorios debio a que el correr constantemente pruebas demandarían mucho tiempo para los entrenamietnos, por lo que se buscó optimzar el tiempo dedicado al entrenamiento enfocandose sólo en los bosques aleatorios que además tambíen eran los más complejos.

|Modelo|AUC-ROC score|
|------|-------------|
|Sin CV| 0.85|
|Con CV|0.86| 



* Bosque aleatorio simple

Se denominó simple debido a que no tuvo un tunning de hiperparámetros posterior utilizando algún framework como GridSearchCV u Optuna. 

|Modelo|AUC-ROC score|
|------|-------------|
|Sin CV | 0.89 |
| Con CV | 0.84 |

La razón por la cual podría explicarse que al utilizar la CV disminuyera el valor de AUC-ROC score podría ser debido a que se este produciendo sobreentrenamiento y que este fenómeno fue más apreciable en el modelo sin CV.



* Bosque aleatorio con tunning de hiperparámetros

Se usaron dos frameworks para optimizar los hiperparámetros: GridSearchCV y Optuna. Se obtuvó un AUC-ROC score ligeramente superior al utilizar Optuna.

|Modelo|AUC-ROC score|
|------|-------------|
|GridSearchCV | 0.87 |
| Optuna | 0.89 |



* Bosque aleatorio con potenciación de gradiente

Se utilizó el framework CatboosClassifier() con un modelos de bosque aleatorio.

|Modelo|AUC-ROC score|
|------|-------------|
|Catboost | 0.89 |



* Bosque aleatorio con potenciación de gradiente y con tunning de hiperparámetros con Optuna

Se eligió el bosque aleatorio con potenciación de gradiente ya que su valor de AUC-ROC score es superior que al utilizar un bosque aleatorio simple con o sin validación cruzada, sin embargo, con base en lo reportado se esperaría que si se usa la validación cruzada la robustes aumentaría por lo cual se conservo esta característica. Al realizar el tunning de hiperparámetros se obtuvó un score más elevado con Optuna por lo cual se consideró para la creación de este modelo.

|Modelo|AUC-ROC score|
|------|-------------|
|Catboost + CV + Optuna | 0.89 |


__Modelo de final__

El realizar las predicciones con el modelo de bosque aleatorio con potenciación de gradiente y tunning de hiperparámetros con Optuna se obtuvo un AUC-ROC score 0.72. La disminución con respecto al conjunto de validación y entrenamiento puede deberse a overfitting debido al sobremuestreo que se utilizó para tratar el desequilibrio de clases en la columna __End_Date__.
